In [1]:
# Imports we need.
import numpy as np
import os
import collections

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0" # change as needed

import tensorflow as tf
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.layers import common_layers
from tensor2tensor.utils import trainer_lib
from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry
from tensor2tensor.utils import metrics

# Enable TF Eager execution
tfe = tf.contrib.eager
tfe.enable_eager_execution()
Modes = tf.estimator.ModeKeys

from define_problem import MimicDischargeSummaries
import subprocess as sp

W0805 23:01:20.714245 139928614676288 deprecation_wrapper.py:119] From /home/aa5118/anaconda3/envs/tf/lib/python3.7/site-packages/tensor2tensor/utils/expert_utils.py:68: The name tf.variable_scope is deprecated. Please use tf.compat.v1.variable_scope instead.

W0805 23:01:21.253370 139928614676288 lazy_loader.py:50] 
The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

W0805 23:01:23.112778 139928614676288 deprecation_wrapper.py:119] From /home/aa5118/anaconda3/envs/tf/lib/python3.7/site-packages/tensor2tensor/utils/adafactor.py:27: The name tf.train.Optimizer is deprecated. Please use tf.compat.v1.train.Optimizer instead.

W0805 23:01:23.113874 139928614676288 deprecation_wrapper

In [2]:
# Fetch the problem
ende_problem = problems.problem("mimic_discharge_summaries")

# Get the encoders from the problem
data_dir = os.path.expanduser("../../data/t2t_experiments/full_context/data/")
encoders = ende_problem.feature_encoders(data_dir)

# Setup helper functions for encoding and decoding
def encode(input_str, output_str=None):
    """Input str to features dict, ready for inference"""
    inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id
    batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
    return {"inputs": batch_inputs}

def decode(integers):
    """List of ints to str"""
    integers = list(np.squeeze(integers))
    if 1 in integers:
        integers = integers[:integers.index(1)]
    return encoders["inputs"].decode(np.squeeze(integers))

W0805 23:01:23.767412 139928614676288 deprecation_wrapper.py:119] From /home/aa5118/anaconda3/envs/tf/lib/python3.7/site-packages/tensor2tensor/data_generators/text_encoder.py:938: The name tf.gfile.Exists is deprecated. Please use tf.io.gfile.exists instead.

W0805 23:01:23.768171 139928614676288 deprecation_wrapper.py:119] From /home/aa5118/anaconda3/envs/tf/lib/python3.7/site-packages/tensor2tensor/data_generators/text_encoder.py:940: The name tf.gfile.Open is deprecated. Please use tf.io.gfile.GFile instead.



In [3]:
# Copy the pretrained checkpoint locally
ckpt_path=u'../../data/t2t_experiments/full_context/output/model.ckpt-5000'

In [4]:
# Create hparams and the model
model_name = "transformer"
hparams_set = "transformer_base"

hparams = trainer_lib.create_hparams(hparams_set, data_dir=data_dir, problem_name="mimic_discharge_summaries")
hparams.max_length=10000
hparams.max_target_seq_length=512
hparams.max_input_seq_length=512

# NOTE: Only create the model once when restoring from a checkpoint; it's a
# Layer and so subsequent instantiations will have different variable scopes
# that will not match the checkpoint.
translate_model = registry.model(model_name)(hparams, Modes.EVAL)

W0805 23:01:24.063292 139928614676288 deprecation_wrapper.py:119] From /home/aa5118/anaconda3/envs/tf/lib/python3.7/site-packages/tensor2tensor/layers/common_layers.py:94: The name tf.get_default_graph is deprecated. Please use tf.compat.v1.get_default_graph instead.

W0805 23:01:24.261667 139928614676288 deprecation_wrapper.py:119] From /home/aa5118/anaconda3/envs/tf/lib/python3.7/site-packages/tensor2tensor/utils/t2t_model.py:243: The name tf.summary.text is deprecated. Please use tf.compat.v1.summary.text instead.



In [5]:
# Restore and translate!
def translate(inputs):
    encoded_inputs = encode(inputs)
    with tfe.restore_variables_on_create(ckpt_path):
        model_output = translate_model.infer(encoded_inputs, beam_size=4, alpha=0.6)["outputs"]
    return decode(model_output)

In [6]:
f = open("../../data/preprocessed/src-test.txt",'r')
line = f.readline()
inputs =""
i=0
while line:
    if i==1:
        break
    inputs+=line+"\n"
    line=f.readline()
    i+=1
print(inputs)

admission date : [ 2164/8/31 ] discharge date : [ <H> M <G> 84 <A> black <E> acute pancreatitis | pneumonia, organism unspecified | malignant neoplasm of duodenum | atrial fibrillation | anemia, unspecified | unspecified essential hypertension | pure hypercholesterolemia | coronary atherosclerosis of native coronary artery | old myocardial infarction | hypertrophy (benign) of prostate without urinary obstruction and other lower urinary tract symptom (luts) | personal history of venous thrombosis and embolism | long-term (current) use of anticoagulants <D>  <P> olanzapine (disintegrating tablet) , 5mg Tablet | lorazepam , 0.5mg Tablet | lorazepam , 0.5mg Tablet | morphine sulfate (oral soln.) , 10mg/5mL UDCUP | metoprolol tartrate , 25mg Tablet | enoxaparin sodium , 80mg Syringe | enoxaparin sodium , 80mg Syringe | lorazepam , 1mg Tablet <M> mrsa screen : None <T>  <L>




In [7]:
inputs = [sp.getoutput('head -1 ../../data/preprocessed/src-test.txt')]
for context in inputs:
    #print (context, "\n")
    print (translate(context))

W0805 23:01:24.981052 139928614676288 deprecation_wrapper.py:119] From /home/aa5118/anaconda3/envs/tf/lib/python3.7/site-packages/tensor2tensor/utils/t2t_model.py:2223: The name tf.get_variable_scope is deprecated. Please use tf.compat.v1.get_variable_scope instead.

W0805 23:01:25.069461 139928614676288 deprecation.py:323] From /home/aa5118/anaconda3/envs/tf/lib/python3.7/site-packages/tensor2tensor/layers/common_attention.py:856: to_float (from tensorflow.python.ops.math_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use `tf.cast` instead.
W0805 23:01:25.077659 139928614676288 deprecation.py:323] From /home/aa5118/anaconda3/envs/tf/lib/python3.7/site-packages/tensor2tensor/utils/expert_utils.py:621: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
W0805 23:01:25.078293 1

KeyboardInterrupt: 